In [ ]:
#1A: show statistics(mean,std,...) for each feature using groupby, make barplots to show distribution
#1B: 1.impute missing values using mean value 2.impute missing values as 0
#1C: re-format the data into patient, time, mood, activity, call..., aggregate by 5-day windows to generate instances for training

In [2]:
import pandas as pd
import matplotlib.pyplot as plt 

In [8]:
rawdata=pd.read_csv("dataset_mood_smartphone.csv")

In [9]:
rawdata

,Unnamed: 0,id,time,variable,value
0,1,AS14.01,2014-02-26 13:00:00.000,mood,6.000
1,2,AS14.01,2014-02-26 15:00:00.000,mood,6.000
2,3,AS14.01,2014-02-26 18:00:00.000,mood,6.000
3,4,AS14.01,2014-02-26 21:00:00.000,mood,7.000
4,5,AS14.01,2014-02-27 09:00:00.000,mood,6.000
...,...,...,...,...,...
376907,2770399,AS14.30,2014-04-11 07:51:16.948,appCat.weather,8.032
376908,2772465,AS14.30,2014-04-19 11:00:32.747,appCat.weather,3.008
376909,2774026,AS14.30,2014-04-26 10:19:07.434,appCat.weather,7.026
376910,2774133,AS14.30,2014-04-27 00:44:48.450,appCat.weather,23.033


In [10]:
rawdata=rawdata.drop('Unnamed: 0',axis=1)

In [34]:
rawdata.groupby('variable')['value'].mean()

variable
activity                  0.115958
appCat.builtin           18.538262
appCat.communication     43.343792
appCat.entertainment     37.576480
appCat.finance           21.755251
appCat.game             128.391615
appCat.office            22.578892
appCat.other             25.810839
appCat.social            72.401906
appCat.travel            45.730850
appCat.unknown           45.553006
appCat.utilities         18.537552
appCat.weather           20.148714
call                      1.000000
circumplex.arousal       -0.098624
circumplex.valence        0.687808
mood                      6.992555
screen                   75.335206
sms                       1.000000
Name: value, dtype: float64

In [35]:
rawdata.groupby('variable')['value'].max()

variable
activity                    1.000
appCat.builtin          33960.246
appCat.communication     9830.777
appCat.entertainment    32148.677
appCat.finance            355.513
appCat.game              5491.793
appCat.office           32708.818
appCat.other             3892.038
appCat.social           30000.906
appCat.travel           10452.615
appCat.unknown           2239.937
appCat.utilities         1802.649
appCat.weather            344.863
call                        1.000
circumplex.arousal          2.000
circumplex.valence          2.000
mood                       10.000
screen                   9867.007
sms                         1.000
Name: value, dtype: float64

In [36]:
rawdata.groupby('variable')['value'].min()

variable
activity                    0.000
appCat.builtin         -82798.871
appCat.communication        0.006
appCat.entertainment       -0.011
appCat.finance              0.131
appCat.game                 1.003
appCat.office               0.003
appCat.other                0.014
appCat.social               0.094
appCat.travel               0.080
appCat.unknown              0.111
appCat.utilities            0.246
appCat.weather              1.003
call                        1.000
circumplex.arousal         -2.000
circumplex.valence         -2.000
mood                        1.000
screen                      0.035
sms                         1.000
Name: value, dtype: float64

In [11]:
mood = rawdata[rawdata['variable']=='mood']

In [14]:
avgmood=mood.groupby(['id','day'])['value'].mean()

In [11]:
rawdata['time']=pd.to_datetime(rawdata['time'])

In [32]:
rawdata=rawdata.drop('Unnamed: 0',axis=1)

In [86]:
rawdata[rawdata['value'].isna()].variable.value_counts()

variable
circumplex.valence    156
circumplex.arousal     46
Name: count, dtype: int64

In [12]:
rawdata.set_index('time',inplace=True)

In [ ]:
scores=rawdata.groupby(['id','variable']).resample('D').sum()['value']

In [47]:
scores=rawdata[rawdata['variable'].isin(['circumplex.arousal','circumplex.valence','mood'])]

In [48]:
scores=scores.groupby(['id','variable']).resample('D').mean()['value']

In [49]:
scores

id       variable            time      
AS14.01  circumplex.arousal  2014-02-26   -0.25
                             2014-02-27    0.00
                             2014-02-28     NaN
                             2014-03-01     NaN
                             2014-03-02     NaN
                                           ... 
AS14.33  mood                2014-05-27    6.20
                             2014-05-28    8.20
                             2014-05-29    7.00
                             2014-05-30    6.80
                             2014-05-31    7.00
Name: value, Length: 3993, dtype: float64

In [67]:
scores=scores.to_frame().reset_index()

In [50]:
usage=rawdata[~rawdata['variable'].isin(['circumplex.arousal','circumplex.valence','mood'])]

In [52]:
usage=usage.groupby(['id','variable']).resample('D').sum()['value']

In [63]:
type(usage)

pandas.core.series.Series

In [66]:
usage=usage.to_frame().reset_index()

In [69]:
raw_resampled=pd.concat([scores,usage])

In [70]:
raw_resampled

,id,variable,time,value
0,AS14.01,circumplex.arousal,2014-02-26,-0.25
1,AS14.01,circumplex.arousal,2014-02-27,0.00
2,AS14.01,circumplex.arousal,2014-02-28,NaN
3,AS14.01,circumplex.arousal,2014-03-01,NaN
4,AS14.01,circumplex.arousal,2014-03-02,NaN
...,...,...,...,...
16461,AS14.33,sms,2014-05-25,1.00
16462,AS14.33,sms,2014-05-26,3.00
16463,AS14.33,sms,2014-05-27,2.00
16464,AS14.33,sms,2014-05-28,1.00


In [71]:
raw_resampled['id_time']=raw_resampled['id']+'_'+raw_resampled['time'].astype(str)

In [72]:
long=raw_resampled.drop(['id','time'],axis=1)

In [73]:
long

,variable,value,id_time
0,circumplex.arousal,-0.25,AS14.01_2014-02-26
1,circumplex.arousal,0.00,AS14.01_2014-02-27
2,circumplex.arousal,NaN,AS14.01_2014-02-28
3,circumplex.arousal,NaN,AS14.01_2014-03-01
4,circumplex.arousal,NaN,AS14.01_2014-03-02
...,...,...,...
16461,sms,1.00,AS14.33_2014-05-25
16462,sms,3.00,AS14.33_2014-05-26
16463,sms,2.00,AS14.33_2014-05-27
16464,sms,1.00,AS14.33_2014-05-28


In [74]:
wide=long.pivot(index='id_time',columns='variable',values='value')

In [75]:
wide=wide.drop(wide[wide['mood'].isna()].index)

In [76]:
wide=wide.drop(wide[wide['screen'].isna()].index)

In [77]:
wide

variable,activity,appCat.builtin,appCat.communication,appCat.entertainment,appCat.finance,appCat.game,appCat.office,appCat.other,appCat.social,appCat.travel,appCat.unknown,appCat.utilities,appCat.weather,call,circumplex.arousal,circumplex.valence,mood,screen,sms
id_time,,,,,,,,,,,,,,,,,,,
AS14.01_2014-03-21,3.083152,3139.218,6280.890,1007.456,49.544,NaN,172.206,239.751,4508.500,915.445,0.000,598.754,NaN,6.0,0.200000,0.2,6.20,17978.907000,0.0
AS14.01_2014-03-22,3.790084,731.429,4962.918,93.324,21.076,NaN,0.000,98.143,439.632,37.305,0.000,117.621,NaN,3.0,0.600000,0.5,6.40,6142.161000,1.0
AS14.01_2014-03-23,2.141117,1286.246,5237.319,94.346,43.403,NaN,0.000,72.823,900.839,0.000,0.000,30.086,30.386,0.0,0.200000,0.8,6.80,6773.832001,0.0
AS14.01_2014-03-24,1.895065,866.956,9270.629,976.971,34.106,NaN,3.010,66.558,3223.626,419.805,0.000,178.732,NaN,10.0,0.800000,0.0,6.00,15047.351001,0.0
AS14.01_2014-03-25,2.360979,1032.768,10276.751,68.206,43.054,NaN,0.000,178.819,1919.471,0.000,235.223,222.893,NaN,0.0,0.500000,0.5,6.75,21475.354999,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
AS14.33_2014-05-26,1.497619,1048.739,1235.537,688.769,NaN,NaN,130.530,235.837,5441.264,0.000,0.000,0.000,NaN,10.0,-0.400000,0.0,5.40,18925.187999,3.0
AS14.33_2014-05-27,0.304902,726.391,2078.403,124.694,NaN,NaN,0.000,142.686,2010.364,0.000,0.000,56.173,NaN,1.0,-0.600000,0.4,6.20,4089.879001,2.0
AS14.33_2014-05-28,2.479228,2551.046,1936.415,614.706,NaN,NaN,357.909,529.946,5361.211,0.000,0.000,30.666,NaN,10.0,0.000000,1.2,8.20,14320.867998,1.0
